코딩도장 내용정리

`asyncio`(Asynchronous I/O)는 비동기 프로그래밍을 위한 모듈이며 CPU작업과 I/O를 병렬로 처리하게 해준다.  
동기(Synchronous)처리는 특정 작업이 끝나면 다음 작업을 처리하는 **순차처리** 방식이고, 비동기(Asynchronous)처리는 여러 작업을 처리하도록 예약한 뒤 작업이 끝나면 결과를 받는 방식.

<img src="https://dojang.io/pluginfile.php/14098/mod_page/content/7/047003.png">

### 네이티브 코루틴 만들기
asyncio를 사용하려면 먼저 `async def`라는 키워드로 **네이티브 코루틴**(제너레이터 기반의 코루틴과 구분)을 만든다. `async def`는 파이썬 3.5 이상부터 사용 가능하다.

In [ ]:
import asyncio

async def hello():                # async def로 네이티브 코루틴 만듦
    print('Hello, World!')
    
loop = asyncio.get_event_loop()    # 이벤트 루프를 얻음
loop.run_until_complete(hello())   # hello가 끝날 때까지 기다림
loop.close()                       # 이벤트 루프를 닫음

`async def`로 hello를 만들고 `asyncio.get_event_loop`함수로 이벤트 루프를 얻는다.  
`loop.run_until_complete(hello())`와 같이 `run_until_complete`에 코루틴 객체를 넣는다.  
`run_until_complete`는 네이티브 코루틴이 이벤트 루프에서 실행되도록 예약하고, 해당 네이티브 코루틴이 끝날 때까지 기다린다.  
그러면 이벤트 루프를 통해서 hello 코루틴이 실행되고 할 일이 끝났으면 loop.close로 이벤트 루프를 닫아준다.

### await로 네이티브 코루틴 실행하기
`await`키워드 뒤에 코루틴 객체, 퓨처 객체, 태스크 객체를 지정하면 해당 객체가 끝날 때까지 기다린 뒤 결과를 반환한다. `await`키워드 역시 파이썬 3.5부터 사용 가능하고 3.4에서는 `yield from`을 사용한다.  
```
변수 = await 코루틴객체
변수 = await 퓨처객체
변수 = await 태스크객체
```
주의할 점은 await는 네이티브 코루틴 안에서만 사용할 수 있다는 것.  
    
**참고: 퓨처와 태스크**  
- 퓨처(asyncio.Future)
    - 미래에 할 일을 표현하는 클래스.
    할 일을 취소하거나 상태 확인, 완료 및 결과 설정에 사용
- 태스크(asyncio.Task)
    - asyncio.Future의 파생 클래스
    - asyncio.Future의 기능과 실행할 코루틴의 객체를 포함한다.
    - 코루틴의 실행을 취소하거나 상태 확인, 완료 및 결과 설정에 사용
    
    
**파이썬 3.4 이하에서 asyncio**  
`async def`와 `await`는 파이썬 3.5에서 추가됐다.  
파이썬 3.4에서는 `@asyncio.coroutine`데코레이터로 네이티브 코루틴을 만든다.  
```python
import asyncio

@asyncio.coroutine
async def 함수이름():
    코드
```

두 수를 더하는 네이티브 코루틴을 만들고 코루틴에서 1초 대기한 뒤 결과를 반환해보자

In [ ]:
import asyncio

async def add(a, b):
    print(f'add: {a} + {b}')
    await asyncio.sleep(1.0)   # 1초 대기. asyncio.sleep도 네이티브 코루틴이다
    return a + b               # 두 수를 더한 결과 반환

async def print_add(a, b):
    result = await add(a, b)   # await로 다른 네이티브 코루틴 실행하고 반환값을 변수에 저장
    print(f'print_add: {a} + {b} = {result}')
    
loop = asyncio.get_event_loop()            # 이벤트 루프를 얻음
loop.run_until_complete(print_add(1, 2))   # print_add가 끝날 때까지 이벤트 루프를 실행
loop.close()

`add: 1 + 2`가 출력되고 1초 뒤에 `print_add: 1 + 2 = 3`이 출력된다.  
`print_add`에서는 `await`로 `add`를 실행하고 반환값을 변수 result에 저장한다.  
이렇게 코루틴 안에서 다른 코루틴을 실행할 때는 await를 사용한다.  
`add`에서는 1초 대기 후 두 수를 더한 결과 반환.

### 비동기로 웹 페이지 가져오기
먼저 asyncio를 사용하지 않고 웹 페이지를 순차적으로 가져와보자.  
`urllib.request`의 `urlopen`으로 웹 페이지를 가져온 뒤 웹 페이지의 길이를 출력.  

In [4]:
from time import time
from urllib.request import Request, urlopen

urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]

begin = time()
result = []
for url in urls:
    request = Request(url, headers={'User-Agent': "Mozilla/5.0"})
    response = urlopen(request)
    page = response.read()
    result.append(len(page))
    
print(result)
end = time()
print(f'실행 시간: {end - begin:.3f}초')

[367793, 63559, 87324, 116661, 334445, 118386]
실행 시간: 8.419초


여기서는 urls에 저장된 순서대로 주소에 접근해서 웹 페이지를 가져오도록 만들었다.  
이러면 웹 페이지 하나를 완전히 가져온 뒤에 다음 웹 페이지를 가져와야 하기 때문에 비효율적이다.

asyncio를 사용해서 비동기로 실행해보자.

In [ ]:
from time import time
from urllib.request import Request, urlopen
import asyncio
 
urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]
 
async def fetch(url):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})    # UA가 없으면 403 에러 발생
    response = await loop.run_in_executor(None, urlopen, request)    # run_in_executor 사용
    page = await loop.run_in_executor(None, response.read)           # run in executor 사용
    return len(page)
 
async def main():
    futures = [asyncio.ensure_future(fetch(url)) for url in urls]
                                                           # 태스크(퓨처) 객체를 리스트로 만듦
    result = await asyncio.gather(*futures)                # 결과를 한꺼번에 가져옴
    print(result)

begin = time()
loop = asyncio.get_event_loop()          # 이벤트 루프를 얻음
loop.run_until_complete(main())          # main이 끝날 때까지 기다림
loop.close()                             # 이벤트 루프를 닫음
end = time()
print(f'실행 시간: {end - begin:.3f}초')

1초대로 실행시간이 줄어든다!!!!!!!

main에서 네이티브 코루틴 여러 개를 동시에 실행하는데, 이때 먼저 asyncio.ensure_future함수를 사용하여 태스크(asyncio.Task)객체를 생성하고 리스트로 만들어 준다.  
`태스크객체 = asyncio.ensure_future(코루틴객체 또는 퓨처객체)`  
그 다음 태스크 리스트를 `asyncio.gather`함수에 넣어준다. 이 함수는 모든 코루틴 객체(퓨처, 태스크 객체)가 끝날 때까지 기다린 뒤 결과(반환값)를 리스트로 반환한다.  
`변수 = await asyncio.gather(코루틴객체1, 코루틴객체2)`  
`asyncio.gather`는 리스트가 아닌 위치 인수로 객체를 받으므로 태스크 객체를 리스트로 만들었다면 `asyncio.gather(*futures)`와 같이 리스트를 언패킹해서 넣어준다.  
또 `asyncio.gather`도 코루틴이므로 `await`로 실행한 뒤 결과를 가져온다.  
이때 `asyncio.gather`에 퓨처 객체를 넣은 순서와 결과 리스트에서 요소의 순서는 일치하지 않을 수도 있다.

웹 페이지를 순서대로 가져올 때와 `asyncio`를 사용하여 비동기로 가져올 때를 비교해보자. 비동기부분은 간략화한 개념도이며 실제 실행 과정은 상당히 복잡하다.  
<img src="https://dojang.io/pluginfile.php/14098/mod_page/content/7/047004.png">

**run_in_executor에 키워드 인수를 사용하는 함수 넣기**  
run_in_executor같은 함수는 위치 인수만 넣을 수 있는데 파이썬에서는 키워드 인수를 많이 사용한다.  
run_in_executor에 키워드 인수를 사용하는 함수를 넣을 때는 `functools.partial`을 사용해야 한다. 이 함수는 이름 그대로 부분 함수를 만들어주는 기능을 한다.  
`functools.partial(함수, 위치인수, 키워드인수)`
```python
import functools

async def hello(executor):
    await loop.run_in_executor(None, functools.partial(print, 'Hello', 'Python', end=' '))
```
`functools.partial`은 인수가 포함된 부분 함수를 반환하는데, 반환된 함수에 다시 인수를 지정해서 호출할 수 있다.  
```python
>>> import functools
>>> hello = functools.partial(print, 'Hello', 'Python', end=' ')
>>> hello()
Hello Python
>>> hello('Script', sep='-')   # 부분 함수에 다시 'Script'와 set='-'를 넣어서 호출
Hello-Python-Script
```

### async with과 async for 사용하기
`async with`은 클래스나 함수를 비동기로 처리한 뒤 결과를 반환하는 문법이다.  
`async for`는 비동기로 반복하는 문법.

#### async with
`async with`은 `with` 다음에 클래스의 인스턴스를 지정하고 as 뒤에 결과를 저장할 변수를 지정한다.  
파이썬 3.5부터 사용 가능하다.  
```python
async with 클래스() as 변수:
    코드
```
async with으로 동작하는 클래스를 만들려면 `__aenter__`와 `__aexit__`메서드를 구현해야 한다. (asynchronous enter, asynchronous exit라는 뜻)  
그리고 메서드를 만들 때 반드시 async def를 사용한다.  
```python
class 클래스이름:
    async def __aenter__(self):
        코드
        
    async def __aexit__(self, exc_type, exc_value, traceback):
        코드
```
  
이를 이용해서 1초 뒤에 덧셈 결과를 반환하는 클래스를 만들어보자

In [ ]:
import asyncio

class AsyncAdd:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    async def __aenter__(self):
        await asyncio.sleep(1.0)
        return self.a + self.b  # __aenter__에서 값을 반환하면 as에 지정한 변수에 들어감
    
    async def __aexit__(self, exc_type, exc_value, traceback):
        pass
    
async def main():
    async with AsyncAdd(1, 2) as result: # async with에 클래스의 인스턴스 지정
        print(result)
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

`__aenter__`메서드에서 1초 대기한 뒤 `self.a`와 `self.b`를 더한 결과를 반환하도록 만든다.  
이렇게 `__aenter__`에서 값을 반환하면 as에 지정한 변수에 들어간다.  
`__aexit__`메서드는 `async with as`를 완전히 벗어나면 호출되는데 여기서는 특별히 만들 부분이 없어서 pass(아예 없으면 에러 발생)

#### async for
`async for`로 동작하는 클래스를 만들려면 `__aiter__`와 `__anext__`메서드를 구현해야 한다.(asynchronous iter, asynchronous next)  
마찬가지로 메서드를 만들 때 반드시 `async def`를 사용.  
다음은 1초마다 숫자를 생성하는 비동기 이터레이터다.

In [ ]:
import asyncio

class AsyncCounter:
    def __init__(self, stop):
        self.current = 0
        self.stop = stop
        
    def __aiter__(self):
        return self
    
    async def __anext__(self):
        if self.current < self.stop:
            await asyncio.sleep(1.0)
            r = self.current
            self.current += 1
            return r
        else:
            raise StopAsyncIteration
            
async def main():
    async for i in AsyncCounter(3):   # for 앞에 async를 붙임
        print(i, end=' ')
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

메서드가 `__anext__`, `__aiter__`라는 점만 다를 뿐 일반적인 이터레이터 생성법과 같다.  
끝날 때는 `StopAsyncIteration`예외를 발생시키면 된다.

**제너레이터 방식으로 비동기 이터레이터 만들기**  
`yield`를 사용하여 제너레이터 방식으로 비동기 이터레이터를 만들 수도 있다.  
`async def`로 네이티브 코루틴을 만들고 `yield`를 사용하여 값을 바깥으로 전달하면 된다.(파이썬 3.6이상부터 가능)

import asyncio

async def async_counter(stop):
    n = 0
    while n < stop:
        yield n
        n += 1
        await asyncio.sleep(1.0)
        
async def main():
    async for i in async_counter(3):
        print(i, end=' ')
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

**비동기 표현식**  
`async`와 `await`를 사용하면, 리스트, 딕셔너리, 셋, 제너레이터 표현식을 비동기 표현식으로 만들 수 있다. (파이썬 3.6 이상부터 가능)  
```
리스트: [변수 async for 변수 in 비동기이터레이터()]
딕셔너리: {키: 값 async for 키, 값 in 비동기이터레이터()]
셋: {변수 async for 변수 in 비동기이터레이터()}
제너레이터: (변수 async for 변수 in 비동기이터레이터()}
```
```python
async def main():
    a = [i async for i in AsyncCounter()]
    print(a)
```
다음과 같이 표현식 안에서 `await`로 코루틴을 실행할 수도 있다.  
```python
async def async_one():
    return 1

async def main():
    coroutines = [async_one, async_one, async_one]
    a = [await co() for co in coroutines]
    print(a)
```

---

https://wikidocs.net/21046 내용 정리

웹페이지를 불러오는 함수가 있는데 1000페이지를 불러오기 위해 한 페이지 불러오고, 또 한 페이지 불러오고 하는 식으로 비효율적으로 불러오지 않고 다음과 같이 효율적으로 불러올 수 있을 것이다.
1. 1000페이지를 동시에 불러온다.
2. 한 페이지를 로딩하면서 기다리는 동안 다른 페이지를 불러온다.  
  
비슷해보이지만 일 처리하는 방식은 다르다. 1번이 **멀티스레드**방식이고, 2번은 **비동기 프로그래밍**방식이다.  
비동기식 프로그래밍을 하기 위해서는 함수를 멀티태스킹이 가능한 함수로 만들어야 한다.  
이런 멀티태스킹이 가능한 함수를 특별히 **코루틴(Coroutine)**이라 부른다.  
  
비동기식으로 블로그 포스트를 크롤링 하는 예제를 보자.  
여러 페이지를 로딩될 때까지 기다리지 않고 바로 다음 페이지를 로딩하여 아주 빠르게 모든 페이지를 불러온다.

In [ ]:
import requests
import asyncio
from bs4 import BeautifulSoup
import time
s = time.time()
results = []

#---------------------- A 파트 ------------------------
async def getpage(url):
    req = await loop.run_in_executor(None, requests.get,
                                                               url)
    html = req.text
    soup = await loop.run_in_executor(None, BeautifulSoup,
                            html, 'lxml')
    return soup

#---------------------- B 파트 ------------------------ 
async def main():
    urls = ["https://wp.me/p9x2W1-x",
            "https://wp.me/p9x2W1-w",
            "https://wp.me/p9x2W1-t",
            "https://wp.me/p9x2W1-q",
            "https://wp.me/p9x2W1-p",
            "https://wp.me/p9x2W1-j",
            "https://wp.me/p9x2W1-h"]

    fts = [asyncio.ensure_future(getpage(u)) for u in urls]
    r = await asyncio.gather(*fts)
    global results
    results = r

#---------------------- C 파트 ------------------------
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close
e = time.time()

print("{0:.2f}초 걸렸습니다".format(e - s))

#결과
#0.78초 걸렸습니다

이 코드는 항상 Blocking 방식 크롤링보다 훨씬 빠르다.  
코드는 C 파트가 먼저 실행되고 B파트가 다음, A파트가 그 다음 실행된다.  
  
C 파트에서 비동기식은 기다림 없이 막히면 다음을 실행한다. 그래서 계속 루프가 돌아야 하고 이는 그런 루프를 만드는 과정이다.  
loop 객체를 하나 만들고 main 코루틴이 완료될 때까지 계속 루프가 돌게 된다.

B 파트에서는 async 함수를 만들기 위해 def 앞에 async 키워드를 붙여줬다.  
그래야 이 함수가 코루틴이 된다. 비동기식 프로그래밍에선 함수가 아닌 코루틴이라 부른다.  
fts변수에서 주소들을 future 객체로 만들어 넣었다. future는 계획서라 볼 수 있다.  
주소들이 getpage()코루틴에 의해 실행되는 계획이 리스트로 담기는 것이다.  
실행은 그 다음줄 `await.asyncio.gather(*fts)`에서 실행되고 결과는 r변수에 저장된다.

A 파트를 보면 실제 페이지를 불러오는 코루틴이다.  
`await loop.run_in_executor()`를 보면 일단 `await`는 코루틴을 실행하는 예약어이다.  
이것을 반드시 사용해야 다른 코루틴을 실행할 수 있다.  
실행한 코루틴은 `loop.run_in_executor()`인데 이는 아주 특별한 코루틴이다.  
원래 코루틴으로 만들어지지 않았기 때문에 파이썬에 있는 모든 함수가 비동기식으로 작동될 수 있는 것은 아니다. 하지만 `loop.run_in_executor()`코루틴을 사용하여 일반 함수를 코루틴처럼 실행할 수 있다.  
전체적으로 getpagee() 코루틴은 결국 url을 인자로 받아 페이지를 불러온다.

https://blog.humminglab.io/python-coroutine-programming-1/  
https://blog.humminglab.io/python-coroutine-programming-2/  
여기 내용 정말 좋다!